In [3]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
from libra_py import ERGO_methods
from libra_py import units


/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator

In [ ]:
def run(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline
%sEOF
basis = "STO-3G"
use_simple_starting_guess=1
scf.create_mtx_files_D = 1
scf.create_mtx_files_F = 1
scf.create_mtx_file_S = 1
XC.sparse_mode = 1
scf.store_all_eigenvalues_to_file = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

In [ ]:
EXE = "/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo"
R = ERGO_methods.xyz_traj2gen_sp("md1.xyz", 0)
command = run(EXE, R)
print command
os.system("%s" % (command))

In [10]:
def get_mtx_matrices(filename, act_sp1=None, act_sp2=None):
    """Get the matrices printed out by the DFTB+

    Args: 
        filename ( string ): the name of the file to read. In the MatrixMarket (mtx) format
        act_sp1 ( list of ints or None): the row active space to extract from the original files
            Indices here start from 0. If set to None - the number of AOs will be
            determined automatically from the file. [default: None]
        act_sp2 ( list of ints or None): the cols active space to extract from the original files
            Indices here start from 0. If set to None - the number of AOs will be
            determined automatically from the file. [default: None]
    
    Returns: 
        list of CMATRIX(N, M): X: where N = len(act_sp1) and M = len(act_sp2) 
            These are the corresponding property matrices (converted to the complex type)
    
    """

    
    # Copy the content of the file to a temporary location
    f = open(filename, "r")
    A = f.readlines()    
    f.close()    

    sz = len(A)       
    start = 0
    for i in xrange(sz):
        if A[i][0]!="%":
            start = i        
            break
        
    # Initialize the matrix dimentions
    tmp = A[start].split()
    print tmp
    N = int(float(tmp[0]))
    M = int(float(tmp[1]))
    
    
    if act_sp1!=None:
        N = len(act_sp1)
    if act_sp2!=None:
        M = len(act_sp2)
        
    X = CMATRIX(N, M)
    
        
    for n in xrange(start+1, sz):
        tmp = A[n].split()        
                
        if len(tmp)==3:
            i = int(float(tmp[0])) - 1
            j = int(float(tmp[1])) - 1
            x = float(tmp[2])
        
            
            X.set(i,j, x*(1.0+0.0j))
            X.set(j,i, x*(1.0+0.0j))
        
        
    return X

In [11]:
F = get_mtx_matrices("F_matrix_11.mtx")
F.show_matrix("F.txt")

['6', '6', '21']


In [ ]:
def do_step(i, EXE, norbs):
    
    # normal size of the active space (AO basis)
    act_sp = range(0,norbs)  
    
    # Make an input file for SP calculations 
    DFTB_methods.xyz_traj2gen_sp("md.xyz", "x1.gen", i, "C")

    # Run SCF calculations and generate the charge density for a converged calculations
    # The file x1.gen is used as a geometry
    os.system("cp dftb_in_ham1.hsd dftb_in.hsd")
    os.system( "%s" % EXE )

    # Just generate the Hamiltonian corresponding to the converged density matrix
    os.system("cp dftb_in_ham2.hsd dftb_in.hsd")
    os.system( "%s" % EXE )

    # Get the Hamiltonian    
    Hi = DFTB_methods.get_dftb_matrices("hamsqr1.dat", act_sp, act_sp)    
    Si = DFTB_methods.get_dftb_matrices("oversqr.dat", act_sp, act_sp) 
    Ei = CMATRIX(norbs, norbs)
    MOi = CMATRIX(norbs, norbs)
    solve_eigen(Hi[0], Si[0], Ei, MOi, 0)  # gamma-point only at time i
    
    return Ei, MOi


def do_ovlp(i, EXE, norbs):
        
    # Make an input file for the overlap calculations 
    DFTB_methods.xyz_traj2gen_ovlp("md.xyz", "x2.gen", i, i+1, "C")
    
    # Run SCF calculations and generate the charge density for a converged calculations
    # The file x2.gen is used as a geometry
    os.system("cp dftb_in_overlaps.hsd dftb_in.hsd")
    os.system( "%s" % EXE )

    # Get the Hamiltonian    
    act_sp = range(0,2*norbs)  
    Sbig = DFTB_methods.get_dftb_matrices("oversqr.dat", act_sp, act_sp)    
    
    act_sp1 = range(0,norbs)
    act_sp2 = range(norbs,2*norbs)
    S = CMATRIX(norbs, norbs)
    pop_submatrix(Sbig[0], S, act_sp1, act_sp2)
    
    return S
    
    
    
def run_step2(nsteps, EXE, norbs, dt):

    U_curr = CMATRIX(norbs, norbs)
    U_next = CMATRIX(norbs, norbs)
    E_curr = CMATRIX(norbs, norbs)
    E_next = CMATRIX(norbs, norbs)
        
    act_sp1 = range(0,norbs)  # normal size of the active space (AO basis)
    
    E_curr, U_curr = do_step(0, EXE, norbs)
    
    for i in xrange(1,nsteps-1):
        E_next, U_next = do_step(i, EXE, norbs)
        S = do_ovlp(i, EXE, norbs)
        
        TDM = U_curr.H() * S * U_next
        Hvib = 0.5*(E_curr + E_next) - (0.5j/dt) * ( TDM - TDM.H() )
        
        
        Hvib.real().show_matrix("res/hvib_%i_re" % (i) )
        Hvib.imag().show_matrix("res/hvib_%i_im" % (i) )
        
        # Current becomes the old 
        E_curr = CMATRIX(E_next)
        U_curr = CMATRIX(U_next)
        

In [ ]:
EXE = "/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo"
nsteps = 5
norbs = 87
dt = 1.0 * units.fs2au


os.system("mkdir res")
run_step2(nsteps, EXE, norbs, dt)